In [ ]:
!pip install -U fashion-clip

### data

#### load raw data

In [1]:
import pandas as pd

item_data = pd.read_csv("./data/articles.csv")
interaction_data = pd.read_csv("./data/transactions_train.csv")
user_data = pd.read_csv("./data/customers.csv")

#### img prepare

In [2]:
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import numpy as np

torch.manual_seed(42)
np.random.seed(42)

def img_by_id(df, article_id:int, no_list:list, echo:int=1, img_show:bool=True):
    if article_id in no_list:
        return
    if echo:
        display(df[df.article_id == article_id])

    img_id = "0"+str(article_id)
    img = Image.open("./data/images/"+img_id[0:3]+"/"+img_id+".jpg")

    if img_show:
        img.show()

def find_no_img_item(df):
    no_img = []

    for item in tqdm(df.iterrows(), total=len(df)):
        try:
            img_by_id(df, item[1][0], no_list=no_img, echo=0, img_show=False)
        except FileNotFoundError:
            no_img.append(item[0])

    return no_img

In [3]:
no_img_ids = find_no_img_item(item_data)

100%|██████████| 105542/105542 [00:25<00:00, 4080.37it/s]


In [4]:
len(no_img_ids)

442

In [5]:
no_img_article_id = [item_data.iloc[x].article_id for x in no_img_ids]

n_item_data = item_data.drop(no_img_ids, axis=0).reset_index(drop=True)
n_interaction_data = interaction_data[~interaction_data["article_id"].isin(no_img_article_id)].reset_index(drop=True)

user2idx = {v:k for k,v in enumerate(user_data['customer_id'].unique())}
item2idx = {v:k for k,v in enumerate(n_item_data['article_id'].unique())}

In [6]:
from torchvision.models import alexnet, AlexNet_Weights, resnet18, ResNet18_Weights, vgg16, VGG16_Weights
from fashion_clip.fashion_clip import FashionCLIP


# # load pretrained alexnet
# model_alex = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
# model_res = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
# model_vgg = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

# # del last clf layer
# model_alex.classifier = model_alex.classifier[:-3]
# model_res.fc = nn.Identity()
# model_vgg.classifier = model_vgg.classifier[:-3]

fclip = FashionCLIP('fashion-clip')

images = ["./data/images/" + "0" + str(k)[0:2] + "/" + "0"+str(k) + ".jpg" for k in n_item_data["article_id"].tolist()]
# image_embeddings = fclip.encode_images(images, batch_size=32)


# feat_map_res = make_feature_map(model_res,n_item_data, book2idx)
# feat_map_alex = make_feature_map(model_alex, n_book_data, book2idx)
# feat_map_vgg = make_feature_map(model_vgg,n_book_data, book2idx)

In [ ]:
# 만들어둔 임베딩 csv로 저장해두기
# pd.DataFrame(images).to_csv("img_list.csv", index=False)
# pd.DataFrame(image_embeddings).to_csv("img_emb.csv", index=False)


In [28]:
img_list = pd.read_csv("img_list.csv")
img_emb = pd.read_csv("img_emb.csv")

In [49]:
print(img_emb.shape)

torch.Size([105100, 512])


In [29]:
img_emb = torch.tensor(img_emb.values)

In [52]:
# img 16="0118458003", 17="0118458004"
res = nn.functional.cosine_similarity(img_emb[16], img_emb[17], dim=0)
res

tensor(0.9380, dtype=torch.float64)

#### make custom dataset

In [54]:
import torch
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split

torch.manual_seed(42)
np.random.seed(42)

class HMDataset(Dataset):
    def __init__(self, df, user2idx, item2idx) -> None:
        super().__init__()
        self.df = df
        self.user2idx = user2idx
        self.item2idx = item2idx
        self.n_user = len(self.user2idx)
        self.n_item = len(self.item2idx)
        # mapping id2idx
        self.df['article_id'] = self.df['article_id'].map(self.item2idx)
        self.df['customer_id'] = self.df['customer_id'].map(self.user2idx)
        self.df['neg'] = np.zeros(len(self.df), dtype=int)

        self._make_triples_data()
    
    def __getitem__(self, index):
        user = self.df.customer_id[index]
        pos = self.df.article_id[index]
        neg = self.df.neg[index]
        return user, pos, neg
    
    def _neg_sampling(self, pos_list):
        neg = np.random.randint(0,self.n_item,1)
        while neg in pos_list:
            neg = np.random.randint(0,self.n_item,1)
        return neg

    def _make_triples_data(self):
        for id in tqdm(range(self.n_user)):
            pos_list = (self.df[self.df.customer_id==id].article_id).tolist()
            for i in range(len(self.df[self.df.customer_id==id])):
                idx = self.df[self.df['customer_id'] == id].index[i]
                self.df.at[idx, 'neg'] = self._neg_sampling(pos_list)
    
    def __len__(self):
        return len(self.df)

In [55]:
batch_size = 128
dataset = HMDataset(n_interaction_data, user2idx, item2idx)
train_dataset, test_dataset = random_split(dataset, [0.8,0.2])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

  0%|          | 210/1371980 [01:30<376:41:46,  1.01it/s]

### model

In [ ]:
from torch import nn

class VBPR(nn.Module):
    def __init__(self, n_user, n_item, K, D, F, feature_map) -> None:
        super().__init__()
        self.n_user = n_user
        self.n_item = n_item
        self.K = K
        self.D = D
        self.F = F

        self.offset = nn.Parameter(torch.zeros(1))
        self.user_bias = nn.Embedding(self.n_user,1)
        self.item_bias = nn.Embedding(self.n_item,1)
        self.vis_bias = nn.Embedding(self.F,1)
        self.user_emb = nn.Embedding(self.n_user,self.K)
        self.item_emb = nn.Embedding(self.n_item,self.K)
        self.img_vis_emb = nn.Embedding(self.D, self.F)
        self.user_vis_emb = nn.Embedding(self.n_user, self.D)
        self.feature_map = feature_map
    
        self._init_weights()
    
    def _get_feature_map(self, itemset):
        res = torch.tensor([])
        for item in itemset:
            res = torch.concat((res, self.feature_map[item.item()]), dim=0)
        return res
        
    def _init_weights(self):
        nn.init.xavier_uniform_(self.user_bias.weight)
        nn.init.xavier_uniform_(self.item_bias.weight.data)
        nn.init.xavier_uniform_(self.vis_bias.weight.data)
        nn.init.xavier_uniform_(self.user_emb.weight.data)
        nn.init.xavier_uniform_(self.item_emb.weight.data)
        nn.init.xavier_uniform_(self.img_vis_emb.weight.data)
        nn.init.xavier_uniform_(self.user_vis_emb.weight.data)
    
    def cal_each(self, user, item):
        feat_map = self._get_feature_map(item).T
        vis_term = ((self.user_vis_emb(user))@(self.img_vis_emb.weight@(feat_map))).sum(dim=1) + (self.vis_bias.weight.T)@(feat_map)
        mf_term = self.offset + self.user_bias(user).T + self.item_bias(item).T + (self.user_emb(user)@self.item_emb(item).T).sum(dim=1).unsqueeze(dim=0)
        params = (self.offset, self.user_bias(user), self.item_bias(item), self.vis_bias.weight, self.user_emb(user), self.item_emb(item), self.img_vis_emb.weight, self.user_vis_emb(user))
        return (mf_term+vis_term).squeeze(), params
    
    def forward(self, user, pos, neg):
        xui, pos_params = self.cal_each(user,pos)
        xuj, neg_params = self.cal_each(user,neg)
        return (xui-xuj), pos_params, neg_params


In [ ]:
from torch import nn

class VBPR(nn.Module):
    def __init__(self, n_user, n_item, K, D, F, feature_map) -> None:
        super().__init__()
        self.n_user = n_user
        self.n_item = n_item
        self.K = K
        self.D = D
        self.F = F

        self.offset = nn.Parameter(torch.zeros(1))
        self.user_bias = nn.Embedding(self.n_user,1)
        self.item_bias = nn.Embedding(self.n_item,1)
        self.vis_bias = nn.Embedding(self.F,1)
        self.user_emb = nn.Embedding(self.n_user,self.K)
        self.item_emb = nn.Embedding(self.n_item,self.K)
        self.img_vis_emb = nn.Embedding(self.D, self.F)
        self.user_vis_emb = nn.Embedding(self.n_user, self.D)
        self.feature_map = feature_map
    
        self._init_weights()
    
    def _get_feature_map(self, itemset):
        res = torch.tensor([])
        for item in itemset:
            res = torch.concat((res, self.feature_map[item.item()]), dim=0)
        return res
        
    def _init_weights(self):
        nn.init.xavier_uniform_(self.user_bias.weight)
        nn.init.xavier_uniform_(self.item_bias.weight.data)
        nn.init.xavier_uniform_(self.vis_bias.weight.data)
        nn.init.xavier_uniform_(self.user_emb.weight.data)
        nn.init.xavier_uniform_(self.item_emb.weight.data)
        nn.init.xavier_uniform_(self.img_vis_emb.weight.data)
        nn.init.xavier_uniform_(self.user_vis_emb.weight.data)
    
    def cal_each(self, user, item):
        feat_map = self._get_feature_map(item).T
        vis_term = ((self.user_vis_emb(user))@(self.img_vis_emb.weight@(feat_map))).sum(dim=1) + (self.vis_bias.weight.T)@(feat_map)
        mf_term = self.offset + self.user_bias(user).T + self.item_bias(item).T + (self.user_emb(user)@self.item_emb(item).T).sum(dim=1).unsqueeze(dim=0)
        params = (self.offset, self.user_bias(user), self.item_bias(item), self.vis_bias.weight, self.user_emb(user), self.item_emb(item), self.img_vis_emb.weight, self.user_vis_emb(user))
        return (mf_term+vis_term).squeeze(), params
    
    def forward(self, user, pos, neg):
        xui, pos_params = self.cal_each(user,pos)
        xuj, neg_params = self.cal_each(user,neg)
        return (xui-xuj), pos_params, neg_params


In [ ]:
class BPRLoss(nn.Module):
    def __init__(self, reg_theta, reg_beta, reg_e) -> None:
        super().__init__()
        self.reg_theta = reg_theta
        self.reg_beta = reg_beta
        self.reg_e = reg_e
    
    def _cal_l2(self, *tensors):
        total = 0
        for tensor in tensors:
            total += tensor.pow(2).sum()
        return 0.5 * total

    def _reg_term(self, pos_params, neg_params):
        alpha, beta_u, beta_pos, beta_prime_pos, gamma_u, gamma_pos, e_pos, theta_u = pos_params
        _, _, beta_neg, beta_prime_neg, _, gamma_neg, e_neg, _ = neg_params

        reg_out = self.reg_theta * self._cal_l2(alpha, beta_u, beta_pos, beta_neg, theta_u, gamma_u, gamma_pos, gamma_neg)
        reg_out += self.reg_beta * self._cal_l2(beta_prime_pos, beta_prime_neg)
        reg_out += self.reg_e * self._cal_l2(e_pos, e_neg)

        return reg_out

    def forward(self, diff, pos_params, neg_params):
        loss = -nn.functional.logsigmoid(diff).sum()
        loss += self._reg_term(pos_params, neg_params)

        return loss

In [ ]:
import torch.nn as nn

def train(model, optimizer, dataloader, criterion, device):
    model.train()
    total_loss = 0

    for user, pos, neg in tqdm(dataloader):
        user = user.to(device)
        pos = pos.to(device)
        neg = neg.to(device)

        diff, pos_params, neg_params = model(user, pos, neg)
        loss = criterion(diff, pos_params, neg_params)
        
        model.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    return total_loss/len(dataloader)

In [ ]:
from torch.optim import Adam

n_user = dataset.n_user
n_item = dataset.n_item

K = 20
D = 20
F = 4096
F_res = 512


reg_theta = 0.1
reg_beta = 0.1
reg_e = 0

lr = 0.001
epoch = 20

device = "cuda" if torch.cuda.is_available() else "cpu" 
criterion = BPRLoss(reg_theta, reg_beta, reg_e)

In [ ]:
device

In [ ]:
feat_map = feat_map_alex

vbpr_alex = VBPR(n_user, n_item, K, D, F, feat_map)
optimizer = Adam(params = vbpr_alex.parameters(), lr=lr)
alex_train_loss = []

for i in range(epoch):
    alex_train_loss.append(train(vbpr_alex, optimizer, train_dataloader, criterion, device))
    print(f'EPOCH : {i} | LOSS : {alex_train_loss[-1]:.10}')


In [ ]:
F_res = 512
feat_map = feat_map_res

vbpr_res = VBPR(n_user, n_item, K, D, F_res, feat_map)
optimizer = Adam(params = vbpr_res.parameters(), lr=lr)
res_train_loss = []

for i in range(epoch):
    res_train_loss.append(train(vbpr_res, optimizer, train_dataloader, criterion, device))
    print(f'EPOCH : {i} | LOSS : {res_train_loss[-1]:.10}')

In [ ]:
feat_map = feat_map_vgg

vbpr_vgg = VBPR(n_user, n_item, K, D, F, feat_map)
optimizer = Adam(params = vbpr_vgg.parameters(), lr=lr)
vgg_train_loss = []

for i in range(epoch):
    vgg_train_loss.append(train(vbpr_vgg, optimizer, train_dataloader, criterion, device))
    print(f'EPOCH : {i} | LOSS : {vgg_train_loss[-1]:.10}')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(epoch), alex_train_loss, label="Alexnet")
plt.plot(range(epoch), res_train_loss, label="Resnet")
plt.plot(range(epoch), vgg_train_loss, label="VGG")
plt.legend()

### change n_factor(k)

In [ ]:
K = 40
# D = 20

# reg_theta = 0.1
# reg_beta = 0.1
# reg_e = 0


In [ ]:
vbpr_alex = VBPR(n_user, n_item, K, D, F, feat_map_alex)
optimizer = Adam(params = vbpr_alex.parameters(), lr=lr)
alex_train_loss_40 = []

for i in range(epoch):
    alex_train_loss_40.append(train(vbpr_alex, optimizer, train_dataloader, criterion, device))
    print(f'EPOCH : {i} | LOSS : {alex_train_loss_40[-1]:.10}')

print("--------------------------------------------------------------------")

vbpr_res = VBPR(n_user, n_item, K, D, F_res, feat_map_res)
optimizer = Adam(params = vbpr_res.parameters(), lr=lr)
res_train_loss_40 = []

for i in range(epoch):
    res_train_loss_40.append(train(vbpr_res, optimizer, train_dataloader, criterion, device))
    print(f'EPOCH : {i} | LOSS : {res_train_loss_40[-1]:.10}')

print("--------------------------------------------------------------------")

vbpr_vgg = VBPR(n_user, n_item, K, D, F, feat_map_vgg)
optimizer = Adam(params = vbpr_vgg.parameters(), lr=lr)
vgg_train_loss_40 = []

for i in range(epoch):
    vgg_train_loss_40.append(train(vbpr_vgg, optimizer, train_dataloader, criterion, device))
    print(f'EPOCH : {i} | LOSS : {vgg_train_loss_40[-1]:.10}')

### Top K rec test

In [ ]:
class Recommender:
    def __init__(self, model, query_img, train_dataset, n_item, feature_map, device) -> None:
        self.model = model
        self.train_df = train_dataset.dataset.df
        self.all_item = set(range(0,n_item))
        self.query_img = query_img
        self.feature_map = feature_map
        self.device = device

    def _get_img_sim(self, itemset:list):
        print("GET IMG SIM")
        res = []
        for item in itemset:
            res.append(nn.functional.cosine_similarity(self.query_img, self.feature_map[item.item()]))
        return res

    def _get_unobs_items(self, user_idx):
        obs_item_set = set(self.train_df[self.train_df.user_id==user_idx].isbn)
        return list(self.all_item - obs_item_set)

    def user_rank(self, user_idx:int, top_k:int=None):
        self.model.eval()
        unobs_itemset = self._get_unobs_items(user_idx)

        with torch.no_grad():
            itemset = torch.tensor(unobs_itemset).to(self.device)
            user = torch.tensor(np.full(len(itemset), user_idx)).to(self.device)
            img_sim = torch.tensor(self._get_img_sim(itemset))

            out, _ = self.model.cal_each(user, itemset)
            out = out + img_sim
            scores = np.array(torch.concat((user.unsqueeze(dim=1),itemset.unsqueeze(dim=1),out.unsqueeze(dim=1)), dim=1))
       
        sorted_scores = scores[(-scores[:, 2]).argsort()]
        return sorted_scores[:top_k]

In [ ]:
def eval(recommender, test_dataset):
    df = test_dataset.dataset.df
    user_list = df['user_id'].unique()
    res_true = {}
    res_topk = {}
    res_hit = {}
    
    for user in tqdm(user_list[:50]):
        true_item = df[df.user_id==user].isbn
        if len(true_item)>4:
            res = recommender.user_rank(user, 20)
            topk = res[:,1]
            hit = len(set(true_item).intersection(set(topk)))
            res_true[user] = list(true_item)
            res_topk[user] = list(topk)
            res_hit[user] = hit
    
    return res_true, res_topk, res_hit

In [ ]:
query_isbn = "0440234743"
img = prepare_img(query_isbn)
query = model_res(img)
# res = img_sim(query, feat_map_vgg)
# res
recommender = Recommender(vbpr_res, query, train_dataset, n_item, feat_map_res, device)

res_true, res_topk, res_hit = eval(recommender, test_dataset)